Author: Benny Wai

Update notes: 

In [ ]:
#### to-do's
# make CR - xp table
# rename columns + values with spaces with '_'
# identify

In [2]:
import json
import numpy as np
import pandas as pd

# Read from json

In [3]:
## read the monsters from the SRD
with open('raw_data/srd_5e_monsters.json') as f:
  raw_data = json.load(f)

In [4]:
df = pd.json_normalize(raw_data)

In [16]:
df

,name,meta,Armor Class,Hit Points,Speed,STR,STR_mod,DEX,DEX_mod,CON,...,Challenge,Traits,Actions,Legendary Actions,img_url,Damage Immunities,Condition Immunities,Damage Resistances,Damage Vulnerabilities,Reactions
0,Aboleth,"Large aberration, lawful evil",17 (Natural Armor),135 (18d10 + 36),"10 ft., swim 40 ft.",21,(+5),9,(-1),15,...,"10 (5,900 XP)",<p><em><strong>Amphibious.</strong></em> The a...,<p><em><strong>Multiattack.</strong></em> The ...,"<p>The aboleth can take 3 legendary actions, c...",https://media-waterdeep.cursecdn.com/avatars/t...,NaN,NaN,NaN,NaN,NaN
1,Acolyte,"Medium humanoid, any",10,9 (2d8),30 ft.,10,(+0),10,(+0),10,...,1/4 (50 XP),<p><em><strong>Spellcasting.</strong></em> The...,<p><em><strong>Club.</strong></em> <em>Melee W...,NaN,https://media-waterdeep.cursecdn.com/attachmen...,NaN,NaN,NaN,NaN,NaN
2,Adult Black Dragon,"Huge dragon, chaotic evil",19 (Natural Armor),195 (17d12 + 85),"40 ft., fly 80 ft., swim 40 ft.",23,(+6),14,(+2),21,...,"14 (11,500 XP)",<p><em><strong>Amphibious.</strong></em> The d...,<p><em><strong>Multiattack.</strong></em> The ...,"<p>The dragon can take 3 legendary actions, ch...",https://media-waterdeep.cursecdn.com/avatars/t...,Acid,NaN,NaN,NaN,NaN
3,Adult Blue Dragon,"Huge dragon, lawful evil",19 (Natural Armor),225 (18d12 + 108),"40 ft., burrow 30 ft., fly 80 ft.",25,(+7),10,(+0),23,...,"16 (15,000 XP)",<p><em><strong>Legendary Resistance (3/Day).</...,<p><em><strong>Multiattack.</strong></em> The ...,"<p>The dragon can take 3 legendary actions, ch...",https://media-waterdeep.cursecdn.com/avatars/t...,Lightning,NaN,NaN,NaN,NaN
4,Adult Brass Dragon,"Huge dragon, chaotic good",18 (Natural Armor),172 (15d12 + 75),"40 ft., burrow 30 ft., fly 80 ft.",23,(+6),10,(+0),21,...,"13 (10,000 XP)",<p><em><strong>Legendary Resistance (3/Day).</...,<p><em><strong>Multiattack.</strong></em> The ...,"<p>The dragon can take 3 legendary actions, ch...",https://media-waterdeep.cursecdn.com/avatars/t...,Fire,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Young Green Dragon,"Large dragon, lawful evil",18 (Natural Armor),136 (16d10 + 48),"40 ft., fly 80 ft., swim 40 ft.",19,(+4),12,(+1),17,...,"8 (3,900 XP)",<p><em><strong>Amphibious.</strong></em> The d...,<p><em><strong>Multiattack.</strong></em> The ...,NaN,https://media-waterdeep.cursecdn.com/avatars/t...,Poison,Poisoned,NaN,NaN,NaN
323,Young Red Dragon,"Large dragon, chaotic evil",18 (Natural Armor),178 (17d10 + 85),"40 ft., climb 40 ft., fly 80 ft.",23,(+6),10,(+0),21,...,"10 (5,900 XP)",NaN,<p><em><strong>Multiattack.</strong></em> The ...,NaN,https://media-waterdeep.cursecdn.com/avatars/t...,Fire,NaN,NaN,NaN,NaN
324,Young Silver Dragon,"Large dragon, lawful good",18 (Natural Armor),168 (16d10 + 80),"40 ft., fly 80 ft.",23,(+6),10,(+0),21,...,"9 (5,000 XP)",NaN,<p><em><strong>Multiattack.</strong></em> The ...,NaN,https://media-waterdeep.cursecdn.com/avatars/t...,Cold,NaN,NaN,NaN,NaN
325,Young White Dragon,"Large dragon, chaotic evil",17 (Natural Armor),133 (14d10 + 56),"40 ft., burrow 20 ft., fly 80 ft., swim 40 ft.",18,(+4),10,(+0),18,...,"6 (2,300 XP)",<p><em><strong>Ice Walk.</strong></em> The dra...,<p><em><strong>Multiattack.</strong></em> The ...,NaN,https://media-waterdeep.cursecdn.com/avatars/t...,Cold,NaN,NaN,NaN,NaN


# Clean data

In [5]:
df.columns

Index(['name', 'meta', 'Armor Class', 'Hit Points', 'Speed', 'STR', 'STR_mod',
       'DEX', 'DEX_mod', 'CON', 'CON_mod', 'INT', 'INT_mod', 'WIS', 'WIS_mod',
       'CHA', 'CHA_mod', 'Saving Throws', 'Skills', 'Senses', 'Languages',
       'Challenge', 'Traits', 'Actions', 'Legendary Actions', 'img_url',
       'Damage Immunities', 'Condition Immunities', 'Damage Resistances',
       'Damage Vulnerabilities', 'Reactions'],
      dtype='object')

In [7]:
df.drop(['name','meta'])]

SyntaxError: invalid syntax (<ipython-input-7-83a4e87aec4c>, line 1)

In [8]:
### clean the name column (replacing spaces with '_'s) and break meta into 4 seperate columns

## get name, lower case and _ between words
clean_df = pd.DataFrame(df['name'].str.lower().replace(' ','_'), columns = ['name'])

## split the size and type
clean_df = pd.concat([clean_df,pd.DataFrame(
            pd.DataFrame(
                df['meta'].str.split(', ').tolist()
                , columns = ['creature_info','alignment'])
                    ['creature_info'].str.split(' ').tolist()
                        , columns = ['size','type'])], axis=1)

## create alignment column to be split later, first deal with outliers
clean_df = pd.concat([clean_df,
            pd.DataFrame(pd.DataFrame(
                df['meta'].str.split(', ').tolist()
                , columns = ['creature_info','alignment'])['alignment'])], axis=1)
when_conditions = [clean_df['alignment'] == 'unaligned',clean_df['alignment'] == 'any', clean_df['alignment'] == 'neutral', clean_df['alignment'] == 'any evil alignment']
then_statements = ['unaligned unaligned','any any','neutral neutral','any evil']
clean_df['alignment'] = np.select(when_conditions,then_statements, default=clean_df['alignment'])

## split the alignment column
clean_df = pd.concat([clean_df,
            pd.DataFrame(
                clean_df['alignment'].str.split(' ').tolist()
                , columns = ['ethics','moral'])], axis=1)
clean_df = clean_df.drop(['alignment'], axis = 1)

## lower case all current columns
clean_df = clean_df.apply(lambda x: x.astype(str).str.lower())

clean_df = pd.concat([clean_df,df.drop(['name','meta'],axis=1)],axis=1)
clean_df

,name,size,type,ethics,moral,Armor Class,Hit Points,Speed,STR,STR_mod,...,Challenge,Traits,Actions,Legendary Actions,img_url,Damage Immunities,Condition Immunities,Damage Resistances,Damage Vulnerabilities,Reactions
0,aboleth,large,aberration,lawful,evil,17 (Natural Armor),135 (18d10 + 36),"10 ft., swim 40 ft.",21,(+5),...,"10 (5,900 XP)",<p><em><strong>Amphibious.</strong></em> The a...,<p><em><strong>Multiattack.</strong></em> The ...,"<p>The aboleth can take 3 legendary actions, c...",https://media-waterdeep.cursecdn.com/avatars/t...,NaN,NaN,NaN,NaN,NaN
1,acolyte,medium,humanoid,any,any,10,9 (2d8),30 ft.,10,(+0),...,1/4 (50 XP),<p><em><strong>Spellcasting.</strong></em> The...,<p><em><strong>Club.</strong></em> <em>Melee W...,NaN,https://media-waterdeep.cursecdn.com/attachmen...,NaN,NaN,NaN,NaN,NaN
2,adult black dragon,huge,dragon,chaotic,evil,19 (Natural Armor),195 (17d12 + 85),"40 ft., fly 80 ft., swim 40 ft.",23,(+6),...,"14 (11,500 XP)",<p><em><strong>Amphibious.</strong></em> The d...,<p><em><strong>Multiattack.</strong></em> The ...,"<p>The dragon can take 3 legendary actions, ch...",https://media-waterdeep.cursecdn.com/avatars/t...,Acid,NaN,NaN,NaN,NaN
3,adult blue dragon,huge,dragon,lawful,evil,19 (Natural Armor),225 (18d12 + 108),"40 ft., burrow 30 ft., fly 80 ft.",25,(+7),...,"16 (15,000 XP)",<p><em><strong>Legendary Resistance (3/Day).</...,<p><em><strong>Multiattack.</strong></em> The ...,"<p>The dragon can take 3 legendary actions, ch...",https://media-waterdeep.cursecdn.com/avatars/t...,Lightning,NaN,NaN,NaN,NaN
4,adult brass dragon,huge,dragon,chaotic,good,18 (Natural Armor),172 (15d12 + 75),"40 ft., burrow 30 ft., fly 80 ft.",23,(+6),...,"13 (10,000 XP)",<p><em><strong>Legendary Resistance (3/Day).</...,<p><em><strong>Multiattack.</strong></em> The ...,"<p>The dragon can take 3 legendary actions, ch...",https://media-waterdeep.cursecdn.com/avatars/t...,Fire,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,young green dragon,large,dragon,lawful,evil,18 (Natural Armor),136 (16d10 + 48),"40 ft., fly 80 ft., swim 40 ft.",19,(+4),...,"8 (3,900 XP)",<p><em><strong>Amphibious.</strong></em> The d...,<p><em><strong>Multiattack.</strong></em> The ...,NaN,https://media-waterdeep.cursecdn.com/avatars/t...,Poison,Poisoned,NaN,NaN,NaN
323,young red dragon,large,dragon,chaotic,evil,18 (Natural Armor),178 (17d10 + 85),"40 ft., climb 40 ft., fly 80 ft.",23,(+6),...,"10 (5,900 XP)",NaN,<p><em><strong>Multiattack.</strong></em> The ...,NaN,https://media-waterdeep.cursecdn.com/avatars/t...,Fire,NaN,NaN,NaN,NaN
324,young silver dragon,large,dragon,lawful,good,18 (Natural Armor),168 (16d10 + 80),"40 ft., fly 80 ft.",23,(+6),...,"9 (5,000 XP)",NaN,<p><em><strong>Multiattack.</strong></em> The ...,NaN,https://media-waterdeep.cursecdn.com/avatars/t...,Cold,NaN,NaN,NaN,NaN
325,young white dragon,large,dragon,chaotic,evil,17 (Natural Armor),133 (14d10 + 56),"40 ft., burrow 20 ft., fly 80 ft., swim 40 ft.",18,(+4),...,"6 (2,300 XP)",<p><em><strong>Ice Walk.</strong></em> The dra...,<p><em><strong>Multiattack.</strong></em> The ...,NaN,https://media-waterdeep.cursecdn.com/avatars/t...,Cold,NaN,NaN,NaN,NaN


In [17]:
import urllib.request

In [21]:
urllib.request.urlretrieve(clean_df.loc[0,'img_url'], "monster_images/"+clean_df.loc[0,'name']+".jpeg")

('monster_images/aboleth.jpeg', <http.client.HTTPMessage at 0x20be8040d08>)

In [16]:
clean_df.loc[0,'img_url']

'https://media-waterdeep.cursecdn.com/avatars/thumbnails/0/11/1000/1000/636238825975375671.jpeg'

In [18]:
clean_df.loc[0,'name']

'aboleth'

In [22]:
clean_df['type'].value_counts()

beast          95
dragon         43
humanoid       40
monstrosity    40
fiend          23
undead         20
elemental      16
giant          11
construct       9
plant           7
aberration      7
fey             6
celestial       6
ooze            4
Name: type, dtype: int64

In [26]:
clean_df[0:5]['img_url'].apply(lambda x: urllib.request.urlretrieve(clean_df.loc[0,'img_url'], "monster_images/"+clean_df.loc[0,'name']+".jpeg"))

0    https://media-waterdeep.cursecdn.com/avatars/t...
1    https://media-waterdeep.cursecdn.com/attachmen...
2    https://media-waterdeep.cursecdn.com/avatars/t...
3    https://media-waterdeep.cursecdn.com/avatars/t...
4    https://media-waterdeep.cursecdn.com/avatars/t...
Name: img_url, dtype: object

In [98]:
pd.DataFrame(df['meta'].str.split(', ',0).tolist(), columns = ['creature_info','alignment'])['alignment'].value_counts()

unaligned             125
chaotic evil           46
lawful evil            37
neutral evil           28
any                    23
neutral                22
lawful good            19
chaotic good           12
neutral good            6
lawful neutral          4
chaotic neutral         4
any evil alignment      1
Name: alignment, dtype: int64

In [92]:
pd.DataFrame(pd.DataFrame(
                df['meta'].str.split(', ').tolist()
                , columns = ['creature_info','alignment'])['alignment'])

,alignment
0,lawful evil
1,any
2,chaotic evil
3,lawful evil
4,chaotic good
...,...
322,lawful evil
323,chaotic evil
324,lawful good
325,chaotic evil


In [58]:
df['name'].str.lower()

0                  aboleth
1                  acolyte
2       adult black dragon
3        adult blue dragon
4       adult brass dragon
              ...         
322     young green dragon
323       young red dragon
324    young silver dragon
325     young white dragon
326                 zombie
Name: name, Length: 327, dtype: object

In [55]:
pd.DataFrame(
    pd.DataFrame(
        df['meta'].str.split(', ').tolist()
        , columns = ['creature_info','alignment'])
            ['creature_info'].str.split(' ').tolist()
                , columns = ['size','type'])

,size,type
0,Large,aberration
1,Medium,humanoid
2,Huge,dragon
3,Huge,dragon
4,Huge,dragon
...,...,...
322,Large,dragon
323,Large,dragon
324,Large,dragon
325,Large,dragon


In [65]:
pd.DataFrame(df['meta'].str.split(', ',0).tolist(), columns = ['creature_info','alignment'])['alignment'].value_counts()

unaligned             125
chaotic evil           46
lawful evil            37
neutral evil           28
any                    23
neutral                22
lawful good            19
chaotic good           12
neutral good            6
lawful neutral          4
chaotic neutral         4
any evil alignment      1
Name: alignment, dtype: int64

In [44]:
df['meta']

0      Large aberration, lawful evil
1               Medium humanoid, any
2          Huge dragon, chaotic evil
3           Huge dragon, lawful evil
4          Huge dragon, chaotic good
                   ...              
322        Large dragon, lawful evil
323       Large dragon, chaotic evil
324        Large dragon, lawful good
325       Large dragon, chaotic evil
326      Medium undead, neutral evil
Name: meta, Length: 327, dtype: object